<a href="https://colab.research.google.com/github/Ppangchon/project/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import PIL
import time
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import pickle as p
import plotly
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
import plotly.graph_objs as go
from tensorflow import keras
from tensorflow.keras.models import Sequential

## เตรียมชุดของข้อมูล

In [ ]:
import pathlib
datadir = “D:/data fruit” # path ของ dataset
data = pathlib.Path(datadir)
image_count = len(list(data.glob(‘*/*.jpg’))) # ขนาดของ dataset
print(“จำนวนรูปภาพที่มี .jpg : “,image_count)

In [ ]:
batch_size = 15   # กำหนดขนาดของแต่ละการอ่านข้อมูล
img_height = 150  # ความสูงของรูปภาพแต่ละรูป
img_width = 150   # ความกว้างของรูปภาพแต่ละรูป

## Train

In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(
data,
validation_split=0.2, #sแบ่งข้อมูล เพื่อ training 80% และ validate 20%
subset=”training”,
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size)
val = tf.keras.preprocessing.image_dataset_from_directory(
data,
validation_split=0.2,
subset=”validation”,
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 16))
for images, labels in train.take(1):
for i in range(9):
ax = plt.subplot(3, 3, i + 1)
plt.imshow(images[i].numpy().astype(“uint8”))
plt.title(class_names[labels[i]])
plt.axis(“off”)

In [ ]:
for image_batch, labels_batch in train:
print(image_batch.shape)
print(labels_batch.shape)
break

## Normalization

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

## Model

In [ ]:
num_classes = 5
epochs=10   # จำนวนครั้งที่เราจะ Train

In [ ]:
model = Sequential([
layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
layers.Conv2D(16, 3, padding=’same’, activation=’relu’),
layers.MaxPooling2D(),
layers.Conv2D(32, 3, padding=’same’, activation=’relu’),
layers.MaxPooling2D(),
layers.Conv2D(64, 3, padding=’same’, activation=’relu’),
layers.MaxPooling2D(),
layers.Flatten(),
layers.Dense(128, activation=’relu’),
layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer=’adam’,
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=[‘accuracy’])
model.summary()

In [ ]:
his = model.fit(
train,
validation_data=val,
epochs=epochs
)

## Save Model

In [ ]:
with open(‘history_model’, ‘wb’) as file:
p.dump(his.history, file)

filepath=’model1.h5'
model.save(filepath)
filepath_model = ‘model1.json’
filepath_weights = ‘weights_model.h5’
model_json = model.to_json()
with open(filepath_model, “w”) as json_file:
json_file.write(model_json)

model.save_weights(‘weights_model.h5’)
print(“Saved model to disk”)

## Load Model

In [ ]:
with open(‘history_model’, ‘rb’) as file:
his = p.load(file)

h1 = go.Scatter(y=his[‘loss’],
mode=”lines”, line=dict(
width=2,
color=’blue’),
name=”loss”
)
h2 = go.Scatter(y=his[‘val_loss’],
mode=”lines”, line=dict(
width=2,
color=’red’),
name=”val_loss”
)

data = [h1,h2]
layout1 = go.Layout(title=’Loss’,
xaxis=dict(title=’epochs’),
yaxis=dict(title=’’))
fig1 = go.Figure(data, layout=layout1)
plotly.offline.iplot(fig1, filename=”testMNIST”)
predict_model = load_model(filepath)
predict_model.summary()
with open(filepath_model, ‘r’) as f:
loaded_model_json = f.read()
predict_model = model_from_json(loaded_model_json)
predict_model.load_weights(filepath_weights)
print(“Loaded model from disk”)

## Prediction

In [ ]:
import requests
from IPython.display import Image
from io import BytesIO
test_path = (‘D:/a1.jpg’)
img = keras.preprocessing.image.load_img(
test_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
predictions = predict_model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(“apple”,score[0],”banana”,score[1],”mangosteen”,score[2],”orange”,score[3],”pineapple”,score[4])
display(Image(filename=test_path,width=180, height=180))
if score[0]==np.max(score) :
fruit = “apple”
elif score[1]==np.max(score) :
fruit = “banana”
elif score[2]==np.max(score) :
fruit = “mangosteen”
elif score[3]==np.max(score) :
fruit = “orange”
elif score[4]==np.max(score) :
fruit = “pineapple”
print(
“AI {} มีความมั่นใจ {:.2f}%.”
.format(fruit, 100 * np.max(score))
)